### <font color='#FF93D5'> [0] <u>Initialization</u> </font>

In [ ]:
# remove residues from notebook
!rm -rf sample_data

# get the zip containing the cat images
!wget https://github.com/Crash285github/nyaural_catwork/raw/main/data/cats.zip -P data/

# extract the cats into data/
from zipfile import ZipFile
with ZipFile('data/cats.zip', 'r') as cats:
  cats.extractall('data')

# remove the downloaded zip
!rm data/cats.zip

### <font color='#FF7580'>[1] <u>Data manipulation</u> </font>

#### <font color='#777'> [1.0] Imports, parameters</font>

In [ ]:
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms as tforms
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

# should the images be grayscaled
is_grayscale = True

# how to divide the dataset later on
train_frac, val_frac, test_frac = 7, 2, 1

# show dataset images on [1.3.1]
visualize_datasets = True
# how many images to show
number_of_samples = 5

#### <font color='#C45A63'> [1.1] Dataset class</font>

In [ ]:
class CatDataset(Dataset):
    def __init__(self, main_dir, transform):
        self.main_dir = main_dir
        self.transform = transform
        self.total_imgs = os.listdir(main_dir)

        for img in self.total_imgs:
            if img.endswith(".zip"):
                self.total_imgs.remove(img)

    def __len__(self):
        return len(self.total_imgs)

    def __getitem__(self, idx):
        img_loc = os.path.join(self.main_dir, self.total_imgs[idx])
        image = Image.open(img_loc).convert('RGB')

        tensor_image = self.transform(image)
        return tensor_image

#### <font color='#C45A63'>[1.2] Dataset details</font>

In [ ]:
# image transformations
transforms = tforms.Compose([
  tforms.ToTensor(),
  tforms.Grayscale()
]) if is_grayscale else tforms.Compose([
  tforms.ToTensor(),
])

# normalize divisions
n_train_frac = train_frac / (train_frac + val_frac + test_frac)
n_val_frac = val_frac / (train_frac + val_frac + test_frac)
n_test_frac = test_frac / (train_frac + val_frac + test_frac)

print(f'{(n_train_frac*100):.1f} % Training data')
print(f'{(n_val_frac*100):.1f} % Validation data')
print(f'{(n_test_frac*100):.1f} % Testing data')

#### <font color='#C45A63'>[1.3] Create Datasets</font>

In [ ]:
# create the dataset
dataset = CatDataset('data', transforms)

# split the dataset
train_dataset, val_dataset, test_dataset = random_split(dataset, [n_train_frac, n_val_frac, n_test_frac])

print(f'{len(train_dataset)}\ttraining images')
print(f'{len(val_dataset)}\tvalidating images')
print(f'{len(test_dataset)}\ttesting images')

##### <font color='#777'>[1.3.1] Visualize Datasets </font>

In [ ]:
if visualize_datasets:
   def make_plt(dataset, title):
      fig, _ = plt.subplots(1, number_of_samples)
      fig.set_figwidth(24)
      fig.suptitle(title, fontsize=32)

      selected = np.random.choice(len(dataset), number_of_samples, False)
      for i, ind in enumerate(selected):
         plt.subplot(1, number_of_samples, i+1)

         # we permute: (?, 64, 64) --> (64, 64, ?)
         plt.imshow(dataset[ind].permute(1,2,0), cmap='gray')
         plt.axis('off')

   # training
   make_plt(train_dataset, "Training data")

   # validation
   make_plt(val_dataset, "Validation data")

   # testing
   make_plt(test_dataset, "Testing data")

else:
   print(f"Settable parameter 'visualize_datasets' is False" )


#### <font color='#C45A63'>[1.4] Create DataLoaders </font>

In [ ]:
# training loader
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)

# validation loader
val_dataloader = DataLoader(val_dataset, batch_size=4, shuffle=True)

# testing loader
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=True)

### <font color='#FFEC72'>[2] <u>Define AutoEncoders</u> </font>

#### <font color='#777'> [2.0] Imports</font>

In [ ]:
import torch.nn as nn


#### <font color='#CEBD5C'> [2.1] Dense AutoEncoder Class</font>

In [ ]:
class DenseAutoEncoder(nn.Module):
  def __init__(self):
    super(DenseAutoEncoder, self).__init__()
    self.encoder = nn.Sequential(
      nn.Flatten(), # (N, 64, 64) --> (N, 4096)
      nn.Linear(64*64, 24*24), # (N, 4096) --> (N, 256)
      nn.ReLU(),
      nn.Linear(24*24, 16*16),
      nn.ReLU(),
      nn.Linear(16*16, 4*4),
      nn.ReLU(),
      nn.Linear(4*4, 2*2)    # (N, 4)
    )

    self.decoder = nn.Sequential(
      nn.Linear(2*2, 4*4),
      nn.ReLU(),
      nn.Linear(4*4, 16*16),
      nn.ReLU(),
      nn.Linear(16*16, 24*24),
      nn.ReLU(),
      nn.Linear(24*24, 64*64),
      nn.Sigmoid()
    )

  def forward(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)

    return decoded

#### <font color='#CEBD5C'> [2.2] Sparse AutoEncoder Class</font>

In [ ]:
class SparseAutoEncoder(nn.Module):
  def __init__(self, input_size, hidden_size, learning_rate):
    pass

  def forward(self, x):
    pass

### <font color='#77FF5B'>[3] <u>Training Loop</u> </font>

#### <font color='#777'> [3.0] Imports</font>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

#### <font color='#5EAA4C'> [3.1] Models, parameters</font>

In [ ]:
num_epochs = 10

# run the training on cuda gpu if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Dense model
dense_model = DenseAutoEncoder().to(device)
dense_criterion = nn.MSELoss()
dense_optimizer = optim.Adam(dense_model.parameters(), lr=0.001)

#### <font color='#5EAA4C'> [3.2] Train</font>

In [ ]:
for epoch in range(num_epochs):
  for i, image in enumerate(train_dataloader):
    print(f'\rBatch {i+1}/{len(train_dataloader)}', end='')
    image = image.reshape(-1, 64*64).to(device)

    # forward pass
    reconstructed_image = dense_model(image)
    loss = dense_criterion(reconstructed_image, image)

    # backward pass
    dense_optimizer.zero_grad()
    loss.backward()
    dense_optimizer.step()

  print(f'\nEpoch {epoch+1}/{num_epochs}: Loss {loss.item():.4f}')